In [1]:
import numpy as np　#数値計算統計解析やデータ分析などで、 データ加工や演算をスムーズに行うためのライブラリ
import matplotlib.pyplot as plt　#暗黙的かつ自動的に図形や軸を作成するインターフェース
import pandas as pd #データの読込や並べ替え、欠損値（欠けている項目値）の補完などを行うことができるライブラリ
import random #ランダムな要素や乱数を使う
%matplotlib inline
import seaborn as sns #データの可視化(グラフ作成)を行うPythonライブラリです。 Pythonで有名な可視化ライブラリであるmatplotlibのラッパー関数(内包プログラム)

import requests #HTTP通信ライブラリ
import json　#jsonファイルからの読み込み、データの文字列 ⇔ 辞書型変換、ファイルへの保存を行う
import re #正規表現を行うモジュール





a


In [ ]:
# APIの仕様は
url_path = "https://www.land.mlit.go.jp/webland/api/TradeListSearch?from=20171&to=20175&area=13"
request_result = requests.get(url_path)
data_json = request_result.json()["data"]and
data_pd

In [ ]:
data_from_csv = pd.read_csv("13_Tokyo_20171_20184.csv", encoding='cp932')
data_from_csv.head(5)
data_from_csv["A"].unique()
data_used_apartment = data_from_csv.query('種類 == "中古マンション"')
data_used_apartment.shape
data_used_apartment.isnull().sum()
data_added_dummies["面積（㎡）"] = data_added_dummies["面積（㎡）"].astype(float)
plt.hist(data_added_dummies["取引価格（総額）"])
plt.show()

from sklearn.linear_model import LinearRegression # 線形回帰をする用のモジュールを呼び出し
from sklearn.model_selection import train_test_split # 検証用に学習データとテストデータを分割してくれるモジュールを呼び出し

print(data_from_csv.iloc[0])



In [ ]:
!curl -L http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz -O && tar xzvf ta-lib-0.4.0-src.tar.gz
!cd ta-lib && ./configure --prefix=/usr && make && make install && cd - && pip install ta-lib
!pip install tqdm
!pip install plotly --upgrade

def enable_plotly_in_cell():
    import IPython
    from plotly.offline import init_notebook_mode
    display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
    init_notebook_mode( connected=False )

def calc_techs( df ) :
  import talib as ta
  import numpy as np
  df['Highest20'] = np.append( np.nan ,ta.MAX( df['High'].values ,20 )[:-1] )
  df['Highest10'] = np.append( np.nan ,ta.MAX( df['High'].values ,10 )[:-1] )
  df['Lowest20'] = np.append( np.nan ,ta.MIN( df['Low'].values ,20 )[:-1] )
  df['Lowest10'] = np.append( np.nan ,ta.MIN( df['Low'].values ,10 )[:-1] )
  df['EMA350'] = ta.EMA( df['Close'].values ,350 )
  df['EMA25'] = ta.EMA( df['Close'].values ,25 )
  df['ATR20'] = ta.ATR( df['High'].values ,df['Low'].values ,df['Close'].values ,20 )
  return df

  def set_price_data( start_date ,dict_target_symbols ) :
  import pandas as pd
  from tqdm import tqdm
  df = pd.DataFrame()
  for target_symbol in tqdm( list( dict_target_symbols.keys() ) ) :
    df_raw = get_stooq( target_symbol )
    df_raw = calc_techs( df_raw )
    df_raw.columns = pd.MultiIndex.from_product(  [ [ dict_target_symbols[ target_symbol ] ] ,df_raw.columns ] ,names=[ 'Symbol' ,'Item' ] )
    df = pd.concat( [ df ,df_raw ] ,axis='columns' ,join='outer' )
  df = df.ffill()
  df = df.loc[ start_date: ,: ]
  return df

def plot_chart( df ,target_symbol ,start ,end ) :
  import plotly.graph_objs as go
  layout = dict( title = f'{target_symbol}: {start} ～ {end}'
                ,width = 1200 ,height = 700 ,font = dict( size=15 )
                ,xaxis = dict( title="" ,rangeslider=dict( visible=False ) ,showgrid=True ,tickangle=45 ,anchor="y2" ) #type="Date"
                ,yaxis = dict( domain=[.25 ,1  ] ,title="" )
                ,yaxis2 = dict( domain=[0   ,.25] ,title="" ,zeroline=False ) )
  def add_scatter( fig ,df ,column_name ,**kwargs ) : fig.add_trace( go.Scatter( x=df.index ,y=df[ column_name ] ,name=column_name ,**kwargs ) )
  def add_candle( fig ,df ,**kwargs ) : fig.add_trace( go.Candlestick( x=df.index ,open=df['Open'] ,high=df['High'] ,low=df['Low'], close=df['Close'] ,showlegend=False ,**kwargs ) )

  df_plot = df.loc[ start : end ,: ][ target_symbol ]
  fig = go.Figure( data=[] )
  add_scatter( fig ,df_plot ,'ATR20'     ,line=dict( color='rgba(200,0,0,1)' ,width=1 ) ,yaxis='y2' )
  add_scatter( fig ,df_plot ,'Highest20' ,line=dict( color='rgba(150,150,150,.7)' ,width=1 ) )
  add_scatter( fig ,df_plot ,'Lowest20'  ,line=dict( color='rgba(150,150,150,.7)' ,width=1 ) ,fill="tonexty" ,fillcolor="rgba(120,120,120,.2)" )
  add_scatter( fig ,df_plot ,'Highest10' ,line=dict( color='rgba(100,100,100,.7)' ,width=1 ) )
  add_scatter( fig ,df_plot ,'Lowest10'  ,line=dict( color='rgba(100,100,100,.7)' ,width=1 ) ,fill="tonexty" ,fillcolor="rgba(70,70,70,.1)" )
  add_scatter( fig ,df_plot ,'EMA25'     ,line=dict( color='rgba(200,0,0,1)' ,width=1 ) )
  add_scatter( fig ,df_plot ,'EMA350'    ,line=dict( color='rgba(0,0,200,1)' ,width=1 ) )
  add_candle( fig ,df_plot )

  fig.update_layout( **layout )
  enable_plotly_in_cell()
  fig.show()
  

